# Using the new local simulator

In [1]:
%pip install git+https://github.com/amazon-braket/amazon-braket-simulator-v2-python@ksh/multi

  Cloning https://github.com/amazon-braket/amazon-braket-simulator-v2-python (to revision ksh/multi) to /private/var/folders/r_/pj84gncj4wd4t8h813gv7bz00000gr/T/pip-req-build-5kshnfwa
  Running command git clone --filter=blob:none --quiet https://github.com/amazon-braket/amazon-braket-simulator-v2-python /private/var/folders/r_/pj84gncj4wd4t8h813gv7bz00000gr/T/pip-req-build-5kshnfwa
  Running command git checkout -b ksh/multi --track origin/ksh/multi
  Switched to a new branch 'ksh/multi'
  branch 'ksh/multi' set up to track 'origin/ksh/multi'.
  Resolved https://github.com/amazon-braket/amazon-braket-simulator-v2-python to commit 0bc1d26e040cea721c03d9e570921c429d85b9a4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/amazon-braket/amazon-braket-sdk-python (to revision rmshaffer/multi-sim) to /private/var/folders/r_/pj84gncj4wd4t8h813gv7bz00000gr/T/pip-install-0atkkrpl/am

Note: you may need to restart the kernel to use updated packages.


This tutorial serves as an introduction to the new local simulator for Amazon Braket. This tutorial explains how to use the new local simulator and the performance difference you can expect to see.

## How to set up and use the new local simulator

The new local simulator is available as a Python package, [`amazon-braket-simulator-v2`](https://github.com/amazon-braket/amazon-braket-simulator-v2-python). You can install it locally with `pip`. Then all you need to do is create a `LocalSimulator` object with the `"braket_sv_v2"` (state vector) or `"braket_dm_v2"` backend names to use the new local simulator. The new local simulator supports qubit counts up to 32 (state vector) or 16 (density matrix). Keep in mind larger qubit counts require more memory!

In [2]:
# general imports
import numpy as np
import math
import time

# AWS imports: Import Braket SDK modules
from braket.circuits import Circuit, circuit, Gate, Instruction
from braket.devices import LocalSimulator
import braket.simulator_v2

default_simulator = LocalSimulator("braket_sv")
new_sv_simulator  = LocalSimulator("braket_sv_v2")

[juliapkg] Locating Julia ~1.9, =1.10.0
[juliapkg] Using Julia 1.9.4 at /Users/hyatkath/Software/julia-1.9/julia
[juliapkg] Using Julia project at /Users/hyatkath/Projects/braketvirtualenv/julia_env
[juliapkg] Installing packages:
           julia> import Pkg
           julia> Pkg.add([Pkg.PackageSpec(name="BraketSimulator", uuid="76d27892-9a0b-406c-98e4-7c178e9b3dff", url=raw"https://github.com/amazon-braket/BraketSimulator.jl.git", rev=raw"ksh/parsing"), Pkg.PackageSpec(name="PythonCall", uuid="6099a3de-0909-46bc-b1f4-468b9a2dfc0d")])
           julia> Pkg.resolve()
           julia> Pkg.precompile()


    Updating git-repo `https://github.com/amazon-braket/BraketSimulator.jl.git`
   Resolving package versions...
    Updating `~/Projects/braketvirtualenv/julia_env/Project.toml`
  [76d27892] + BraketSimulator v0.0.1 `https://github.com/amazon-braket/BraketSimulator.jl.git#ksh/parsing`
⌅ [6099a3de] + PythonCall v0.9.19
    Updating `~/Projects/braketvirtualenv/julia_env/Manifest.toml`
  [fbe9abb3] + AWS v1.90.4
  [1c724243] + AWSS3 v0.11.2
  [1520ce14] + AbstractTrees v0.4.5
  [ec485272] + ArnoldiMethod v0.4.0
  [31f734f8] + ArrowTypes v2.3.0
  [67c07d97] + Automa v1.0.3
  [39de3d68] + AxisArrays v0.4.7
  [d1d4a3ce] + BitFlags v0.1.9
  [19504a0f] + Braket v0.9.0
  [76d27892] + BraketSimulator v0.0.1 `https://github.com/amazon-braket/BraketSimulator.jl.git#ksh/parsing`
  [336ed68f] + CSV v0.10.14
  [da1fd8a2] + CodeTracking v1.3.5
  [944b1d66] + CodecZlib v0.7.4
  [861a8166] + Combinatorics v1.0.2
  [34da2185] + Compat v4.15.0
  [f0e56b4a] + ConcurrentUtilities v2.4.1
  [992eb4ea] + Con

## Two simple examples: The GHZ state and Quantum Fourier Transform

We already presented the GHZ example circuit in the [Running quantum circuits on simulators notebook](../getting_started/1_Running_quantum_circuits_on_simulators/1_Running_quantum_circuits_on_simulators.ipynb). Here, we'll compare the performance of the old and new local simulators for this relatively simple circuit. The GHZ state is simple to prepare:

In [3]:
def ghz_circuit(n_qubits: int) -> Circuit:
    """
    Function to return simple GHZ circuit ansatz. Assumes all qubits in range(0, n_qubits-1)
    are entangled.

    :param int n_qubits: number of qubits
    :return: Constructed GHZ circuit
    :rtype: Circuit
    """

    circuit = Circuit()                          # instantiate circuit object
    circuit.h(0)                                 # add Hadamard gate on first qubit

    for ii in range(0, n_qubits-1):
        circuit.cnot(control=ii, target=ii+1)    # apply series of CNOT gates
    return circuit

We will simulate the measurement counts for this circuit on both local simulators. The older local simulator can only simulate up to 18 or so qubits for state-vectors, but the new one can work with substantially more. In this case we will not run up to 32 qubits on the new simulator, because the memory use can become quite substantial. 20 qubits is enough to see that the new simulator can outperform the existing default.

In [4]:
qubit_range = range(5, 21, 5)
n_shots     = 50
ghz_circs   = {}
old_results = {}
new_results = {}
old_durations = {}
new_durations = {}
for num_qubits in qubit_range:
    ghz = ghz_circuit(num_qubits)
    old_start = time.time()
    old_results[num_qubits] = default_simulator.run(ghz, shots=n_shots).result()
    old_stop  = time.time()
    old_durations[num_qubits] = old_stop - old_start
    new_start = time.time()
    new_results[num_qubits] = new_sv_simulator.run(ghz, shots=n_shots).result()
    new_stop  = time.time()
    new_durations[num_qubits] = new_stop - new_start
    ghz_circs[num_qubits] = ghz

for num_qubits in qubit_range:
    print(f"GHZ circuit with {num_qubits} qubits:")
    print(ghz_circs[num_qubits])
    print(f'Old local simulator runtime: {old_durations[num_qubits]}')
    print(f'New local simulator runtime: {new_durations[num_qubits]}')

GHZ circuit with 5 qubits:
T  : │  0  │  1  │  2  │  3  │  4  │
      ┌───┐                         
q0 : ─┤ H ├───●─────────────────────
      └───┘   │                     
            ┌─┴─┐                   
q1 : ───────┤ X ├───●───────────────
            └───┘   │               
                  ┌─┴─┐             
q2 : ─────────────┤ X ├───●─────────
                  └───┘   │         
                        ┌─┴─┐       
q3 : ───────────────────┤ X ├───●───
                        └───┘   │   
                              ┌─┴─┐ 
q4 : ─────────────────────────┤ X ├─
                              └───┘ 
T  : │  0  │  1  │  2  │  3  │  4  │
Old local simulator runtime: 0.057907819747924805
New local simulator runtime: 0.6090741157531738
GHZ circuit with 10 qubits:
T  : │  0  │  1  │  2  │  3  │  4  │  5  │  6  │  7  │  8  │  9  │
      ┌───┐                                                       
q0 : ─┤ H ├───●───────────────────────────────────────────────────
      └───┘   │  

Another example is the quantum Fourier transform (QFT) and its inverse, shown in [the QFT notebook](../advanced_circuits_algorithms/Quantum_Fourier_Transform/Quantum_Fourier_Transform.ipynb). The QFT circuit has more gate operations than GHZ for the same qubit count, so it is a good test to see how efficiently a local simulator implements each gate.

In [5]:
@circuit.subroutine(register=True)
def qft(qubits):    
    """
    Construct a circuit object corresponding to the Quantum Fourier Transform (QFT)
    algorithm, applied to the argument qubits.  Does not use recursion to generate the QFT.
    
    Args:
        qubits (int): The list of qubits on which to apply the QFT
    """
    qftcirc = Circuit()

    # get number of qubits
    num_qubits = len(qubits)
    
    for k in range(num_qubits):
        # First add a Hadamard gate
        qftcirc.h(qubits[k])
    
        # Then apply the controlled rotations, with weights (angles) defined by the distance to the control qubit.
        # Start on the qubit after qubit k, and iterate until the end.  When num_qubits==1, this loop does not run.
        for j in range(1,num_qubits - k):
            angle = 2*math.pi/(2**(j+1))
            qftcirc.cphaseshift(qubits[k+j],qubits[k], angle)
            
    # Then add SWAP gates to reverse the order of the qubits:
    for i in range(math.floor(num_qubits/2)):
        qftcirc.swap(qubits[i], qubits[-i-1])
        
    return qftcirc

In [6]:
qubit_range = range(5, 21, 5)
qft_circs   = {}
old_results = {}
new_results = {}
old_durations = {}
new_durations = {}
for num_qubits in qubit_range:
    # generate QFT circuit
    qft_circ = qft(range(num_qubits))
    old_start = time.time()
    old_results[num_qubits] = default_simulator.run(qft_circ, shots=n_shots).result()
    old_stop  = time.time()
    old_durations[num_qubits] = old_stop - old_start
    new_start = time.time()
    new_results[num_qubits] = new_sv_simulator.run(qft_circ, shots=n_shots).result()
    new_stop  = time.time()
    new_durations[num_qubits] = new_stop - new_start
    qft_circs[num_qubits] = qft

for num_qubits in qubit_range:
    print(f"QFT circuit with {num_qubits} qubits:")
    print(qft_circs[num_qubits])
    print(f'Old local simulator runtime: {old_durations[num_qubits]}')
    print(f'New local simulator runtime: {new_durations[num_qubits]}')

QFT circuit with 5 qubits:
<function qft at 0x103d55990>
Old local simulator runtime: 0.018676042556762695
New local simulator runtime: 0.42661285400390625
QFT circuit with 10 qubits:
<function qft at 0x103d55990>
Old local simulator runtime: 0.03254985809326172
New local simulator runtime: 0.005918979644775391
QFT circuit with 15 qubits:
<function qft at 0x103d55990>
Old local simulator runtime: 0.09259390830993652
New local simulator runtime: 0.22678399085998535
QFT circuit with 20 qubits:
<function qft at 0x103d55990>
Old local simulator runtime: 1.3560099601745605
New local simulator runtime: 0.305574893951416


## Running circuit batches

The new local simulator also has improved support for running *batches* of circuits. To see the effectiveness of this new functionality, we'll run a batch of 5 QFT circuits for varying qubit counts:

In [7]:
qubit_range = range(5, 21, 5)
qft_circs   = {}
old_results = {}
new_results = {}
old_durations = {}
new_durations = {}

batch_size = 5

for num_qubits in qubit_range:
    # generate QFT circuit
    qft_circ = qft(range(num_qubits))
    old_start = time.time()
    batch_circs = [qft_circ for c_ix in range(batch_size)]
    old_results[num_qubits] = default_simulator.run_batch(batch_circs, shots=n_shots).results()
    old_stop  = time.time()
    old_durations[num_qubits] = old_stop - old_start
    new_start = time.time()
    new_results[num_qubits] = new_sv_simulator.run_batch(batch_circs, shots=n_shots).results()
    new_stop  = time.time()
    new_durations[num_qubits] = new_stop - new_start
    qft_circs[num_qubits] = qft

for num_qubits in qubit_range:
    print(f"{batch_size} QFT circuits with {num_qubits} qubits:")
    print(qft_circs[num_qubits])
    print(f'Old local simulator runtime: {old_durations[num_qubits]}')
    print(f'New local simulator runtime: {new_durations[num_qubits]}')

5 QFT circuits with 5 qubits:
<function qft at 0x103d55990>
Old local simulator runtime: 1.9604969024658203
New local simulator runtime: 0.7090916633605957
5 QFT circuits with 10 qubits:
<function qft at 0x103d55990>
Old local simulator runtime: 1.8882651329040527
New local simulator runtime: 0.07212090492248535
5 QFT circuits with 15 qubits:
<function qft at 0x103d55990>
Old local simulator runtime: 15.894394159317017
New local simulator runtime: 0.0876779556274414
5 QFT circuits with 20 qubits:
<function qft at 0x103d55990>
Old local simulator runtime: 28.58506417274475
New local simulator runtime: 0.37873196601867676
